In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import math


import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs_physicsVars_caupi import Mbc_Btag, MM2recoilSignalSide, deltaE

from pandas_plotfuncs import plot_histos_pandas_df, df_call 

In [2]:
# original notebook
# https://naf-jhub.desy.de/user/axelheim/notebooks/private/baumbauen/notebooks/40.7_ah_svenjasCuts_bg_analysis.ipynb

In [3]:
nfs_path = "/nfs/dust/belle2/user/axelheim/thesisPlots_data/"
#corrects_perEvent.to_csv(nfs_path +  "corrects_perEvent.csv")
corrects_perEvent = pd.read_csv(nfs_path +  "corrects_perEvent.csv")

In [4]:
fig, ax = plt.subplots()

#bins = np.linspace(0, 1.25, 40)
bins = np.linspace(0, 2.5, 40)

#fig.suptitle(r'{}'.format("summed_wrongPred_p"))
fig.set_size_inches(8, 5)

ax.set_xlabel(r'summed wrongly assigned momentum / GeV')
ax.set_ylabel("number of events")

for i in range(1,5):
    tmp = corrects_perEvent[corrects_perEvent['num errors']==i]
    N = tmp.shape[0]
    ax.hist(tmp["summed_wrongPred_p"],
            bins=bins,
            density=False, alpha=0.8, label=('{} prediction error per event'.format(i)),histtype='step')

ax.legend()

plt.grid()
plt.savefig('plots/summed_wrongPred_p_svenjasCutsRUN2_oneBG_Btag_histogram.pdf')
#plt.show()


In [5]:
fourMom_summed_preds = pd.read_csv(nfs_path +  "fourMom_summed_preds_chapter7.csv")

In [6]:
fourMom_summed_preds.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'event', 'frac. correct FSPs',
       'num corr preds', 'numFSPs', 'px_summed_bg', 'py_summed_bg',
       'pz_summed_bg', 'E_summed_bg', 'CMSpx_summed_bg', 'CMSpy_summed_bg',
       'CMSpz_summed_bg', 'CMSE_summed_bg', 'px_summed_X', 'py_summed_X',
       'pz_summed_X', 'E_summed_X', 'CMSpx_summed_X', 'CMSpy_summed_X',
       'CMSpz_summed_X', 'CMSE_summed_X', 'Bsig_px', 'Bsig_py', 'Bsig_pz',
       'Bsig_E', 'CMSpx_summed_Bs', 'CMSpy_summed_Bs', 'CMSpz_summed_Bs',
       'CMSE_summed_Bs', 'num errors', 'Mbc_Btag', 'MM2recoilSignalSide',
       'deltaE'],
      dtype='object')

In [7]:
bins = np.linspace(-9,9, 50)
plot_histos_pandas_df(fourMom_summed_preds,"MM2recoilSignalSide",bins,True,'',
                      r'$m_{miss}^2$ / GeV$^2$',
                      "fraction of events",'plots/svenjasCutsRUN2_oneBG_Btag_MM2recoilSignalSide_MbcCut_5.00GeV.pdf',
                      filter_col='Mbc_Btag',greater_than=5.00)

In [8]:
import pylab

tick_factor = 100.0

def numfmt(x, pos): # your custom formatter function: divide by 100.0
    s = '{}'.format(x / tick_factor)
    return s

import matplotlib.ticker as tkr     # has classes for tick-locating and -formatting


fontsize = 16
def plot_histos_pandas_df2(df,col_name,bins,density,title,xlabel,ylabel,save_path,legendpos="right",filter_col='',greater_than=0):
    figsize=(16,10)
    plt.rcParams.update({'font.size': 13})

    fig, ax = plt.subplots()

    fig.set_size_inches(figsize)
    colors = ['b', 'g', 'r', 'y', 'm', 'c', 'k']

    fig.suptitle(title)
    ax.set_xlabel(xlabel, fontsize=fontsize)
    ax.set_ylabel(ylabel, fontsize=fontsize)
    
    
    ax.tick_params(axis='both', which='major', labelsize=fontsize)
    ax.tick_params(axis='both', which='minor', labelsize=fontsize)

    yfmt = tkr.FuncFormatter(numfmt)    # create your custom formatter function

    # your existing code can be inserted here

    pylab.gca().yaxis.set_major_formatter(yfmt)
    
    df_tmp = df_call(df,col_name,filter_col,greater_than)
    mean_all = round(df_tmp.mean(),2)
    std_all = round(df_tmp.std(),2)
    num_all = df_tmp.shape[0]

    n, bins, patches = ax.hist(df[col_name],
            bins=bins,
            density=density, alpha=0.0,histtype='step')

    elem = np.argmax(n)
    peak = round((bins[elem] + bins[elem+1])/2.,2)

    
    ax.hist(df_tmp,
            bins=bins,  
            density=density, alpha=0.5, label=(r'all, $\overline{x}$='+ str(mean_all) + ' GeV' + ', std='+ str(std_all) + ' GeV'  + 
                                               ', peak='+str(peak) + ' GeV' ),color=colors[0],histtype='step')
            #density=density, alpha=0.5, label=(r'all, $\overline{x}$='+ str(mean_all) + ', std='+ str(std_all) + 
                                               #', peak='+str(peak)+', N='+str(num_all)),color=colors[0],histtype='step')


    for i in range(3):
        df_tmp = df_call(df,col_name,filter_col,greater_than,filter_errors=True,error=i)
        mean_filter = round(df_tmp.mean(),2)
        std_filter = round(df_tmp.std(),2)
        num_filter = df_tmp.shape[0]
        
        n, bins, patches = ax.hist(df_tmp,
            bins=bins,
            density=density, alpha=0.0,histtype='step')

        elem = np.argmax(n)
        peak_filter = round((bins[elem] + bins[elem+1])/2.,2)


        ax.hist(df_tmp,
                bins=bins,
                density=density, alpha=0.5, label=(str(i) + r' errors, $\overline{x}$='+ str(mean_filter) + ' GeV'  + 
                        ', std='+ str(std_filter) + ' GeV'  + ', peak='+str(peak_filter) + ' GeV')
                ,color=colors[i+1],histtype='step')

                #', std='+ str(std_filter) + ' GeV'  + ', peak='+str(peak_filter) + ' GeV' +', N='+str(num_filter) )
                #,color=colors[i+1],histtype='step')


    
    #from matplotlib.ticker import FormatStrFormatter
    #ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))


    ax.legend(loc=f'upper {legendpos}')

    plt.savefig(save_path)

In [9]:
bins = np.linspace(4.7,5.3, 50)
plot_histos_pandas_df2(fourMom_summed_preds,"Mbc_Btag",bins,True,'',r'$M_{bc}$ / GeV',
                      "fraction of events",'plots/svenjasCutsRUN2_oneBG_Btag_Mbc_Btag.pdf',legendpos="left")